In [1]:
import matplotlib.pyplot as plt
import os
from project_classes import *
from functions import *
# from iceflow_library import *
# import scipy.optimize as optimize
# from scipy.optimize import curve_fit
print("Imports successful.")

Imports successful.


In [2]:
zoom = True
seg_length = 250
# season = "2009_Antarctica_DC8"
# season1 = "2018_Antarctica_DC8" 
# season2 = "2018_Antarctica_DC8" 
# season1 = "2016_Antarctica_DC8" 
# season2 = "2016_Antarctica_DC8" 
# season = "2014_Antarctica_DC8" 
season1 = "2022_Antarctica_BaslerMKB"
# season2 = "2022_Antarctica_BaslerMKB"
# season1 = "2013_Antarctica_P3"
# flight = "20181030_01"  # the flight date and frame number 
    # that flight only has one point
# flight = "20181018_01"
# flight = "20181103_01"
# flight1 = "20181011_01"
    # one dimensional data error
    # TODO: figure out why 10018 and 1103 have the same data or at least print the same maps and plots
# flight = "20181109_01"
# flight = "20181112_02"  # the problem flight
    # plots fake crossovers along the curved path
# flight = "20161024_05"
# flight1 = "20161015_06"
# flight2 = "20161115_03"
    # probably too close to the coast to be useful
# flight = "20161024_05"
# flight = '20141026_06'

# flight1 = "20230108_01"
# flight2 = "20230109_01"
    # ~ 1/3 of an orbit of the pole and yet the angle plot looks like hot garbage
# file_name = "layer_export_" + flight + ".pickle"
# file_name1 = "C:\\Users\\moser\\Desktop\\cresis_project\\pickle_jar\\layer_export_" + flight1 + ".pickle"
# file_name2 = "C:\\Users\\moser\\Desktop\\cresis_project\\pickle_jar\\layer_export_" + flight2 + ".pickle"
testing = False

In [6]:
# read in the layers from the layer files and save them to a pickle file



In [7]:
def append_layers(layers1, layers2):
    """
    :param layers1: a layer object
    :param layers2: a layer object
    :return: a combined layer object
    """
    print("Appending layers...")
    print("--------------------")
    # append layers2 to layers1
    # layers1 and layers2 have lists of layers. append each list with a given name to the other list with the same name if it exists. if it does not exist, throw an error
    layers = []
    layer_names = []
    elevation = np.array([])
    gps_time = np.array([])
    id = np.array([])
    lat = np.array([])
    lon = np.array([])
    param = np.array([])
    quality = np.array([])
    twtt = np.array([])
    # layer_type = np.array([])
    
    for layer in layers1:
        layer_names.append(layer.layer_name)
        elevation = np.append(elevation, layer.elevation)
        gps_time = np.append(gps_time, layer.gps_time)
        id = np.append(id, layer.id)
        lat = np.append(lat, layer.lat)
        lon = np.append(lon, layer.lon)
        param = np.append(param, layer.param)
        quality = np.append(quality, layer.quality)
        twtt = np.append(twtt, layer.twtt)
        # layer_type = np.append(layer_type, layer.layer_type)
    for layer in layers2:
        if layer.layer_name in layer_names:
            index = layer_names.index(layer.layer_name)
            layers1[index].elevation = np.append(layers1[index].elevation, layer.elevation)
            layers1[index].gps_time = np.append(layers1[index].gps_time, layer.gps_time)
            layers1[index].id = np.append(layers1[index].id, layer.id)
            layers1[index].lat = np.append(layers1[index].lat, layer.lat)
            layers1[index].lon = np.append(layers1[index].lon, layer.lon)
            layers1[index].param = np.append(layers1[index].param, layer.param)
            layers1[index].quality = np.append(layers1[index].quality, layer.quality)
            layers1[index].twtt = np.append(layers1[index].twtt, layer.twtt)
            # layers1[index].layer_type = np.append(layers1[index].layer_type, layer.layer_type)
        else:
            layers1.append(layer)
    return layers1


In [9]:
# print(len(layers1[0].twtt))
# print(len(layers2[0].twtt))
# print(len(layers1[0].twtt) + len(layers2[0].twtt))
# layers1 = append_layers(layers1, layers2)
# print(len(layers1[0].twtt))

# print(layers1)

In [11]:
dir = "C:\\Users\\moser\\Documents\\cresis\\rds\\" + season1 + "\\CSARP_layer"
# print(f"dir: {dir}")
folders = os.listdir(dir)


layers = []
# print(f"flight1: \t{flight1}")
# print(f"folders1:\t{folders[1]}")
for folder in folders:
    # print("folder: ", folder)
    borked = mat_pickler_h5py(season1, folder, testing_mode=testing)  # make it
    if borked:
        debug_print(BRIGHT_RED, f"Folder {folder} is borked. Skipping it.")
        borked = False
        continue        
    else:
        print(f"folder: {folder}")
        layer = read_layers("C:\\Users\\moser\\Desktop\\cresis_project\\pickle_jar\\layer_export_" + folder + ".pickle")
        # print(f"layer: {layer}")
        # print(f"lat shape: {np.shape(layer[0].lat)}")
        # debug_print(f"layer: {layer}")
        # print the shape of the lat-lon arrays
        # debug_print(BRIGHT_YELLOW, f"lat shape: {np.shape(layer[0].lat)}")
    layers = append_layers(layers, layer)
    print(f"size of lon: {len(layer[0].lon)}")


Reading data files...
--------------------
--------------------
folder: 20221229_01
Reading pickle file...
--------------------
Surface
Bottom
--------------------

Appending layers...
--------------------
size of lon: 28
Reading data files...
--------------------
--------------------
folder: 20230106_01
Reading pickle file...
--------------------
Surface
Bottom
--------------------

Appending layers...
--------------------


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (28, 1) + inhomogeneous part.